### Estratégia de Imputação KNN com Ponderação por Eta²

####  Objetivo
Imputar valores ausentes na variável categórica `Type_of_glass`, utilizando as variáveis numéricas como base, com um **KNN personalizado** que utiliza pesos derivados da **associação estatística (Eta²)**.

---

#### Etapas da Abordagem

##### 1. Simulação de valores ausentes:
- Gerar valores ausentes de forma **MCAR** (*Missing Completely At Random*) na coluna `Type_of_glass`.

##### 2. Cálculo do Eta² (*Eta Squared*):
- Para cada variável numérica, calcular o **eta quadrado** em relação a `Type_of_glass`.
- O **eta² quantifica a associação** entre a variável numérica e a categórica.
- Valores mais altos indicam **maior poder preditivo**.

##### 3. Pré-processamento dos dados:
- **Normalizar** todas as variáveis numéricas (ex: Min-Max ou Z-score);
- **Multiplicar** cada variável normalizada pelo seu **eta²**, criando uma **representação ponderada dos dados**.

##### 4. Imputação via KNN personalizado:
Para cada linha com `Type_of_glass` ausente:
- Calcular a **distância** (ex: Euclidiana ou Gower ponderada) entre essa linha e as outras com `Type_of_glass` conhecido;
- Selecionar os **k vizinhos mais próximos**;
- Imputar com a **moda** dos `Type_of_glass` entre esses vizinhos.

---

#### Vantagens da Abordagem
- Leva em conta **quais variáveis são mais relevantes** para prever `Type_of_glass`;
- Melhora a qualidade da imputação comparado a **KNN simples** ou **imputação por moda global**;
- É especialmente útil quando só há **uma variável categórica e muitas numéricas**.


In [54]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import cdist
from collections import Counter
import random

# 1. Carregar e preparar os dados
file_path = 'data/glass.data'
columns = ['Id', 'RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'Type_of_glass']

df = pd.read_csv(file_path, header=None, names=columns)
df.drop(columns=['Id'], inplace=True)

# Salvar os dados originais para avaliação
df_original = df.copy()

random.seed(42)

# 2. Inserir valores ausentes (MCAR) em Type_of_glass
missing_frac = 0.2
n_missing = int(missing_frac * len(df))
missing_indices = random.sample(list(df.index), n_missing)
df.loc[missing_indices, 'Type_of_glass'] = np.nan

# 3. Calcular o Eta² para cada variável numérica em relação a Type_of_glass
def eta_squared(x, y):
    classes = y.dropna().unique()
    grand_mean = x[y.notna()].mean()
    ss_between = sum([
        len(x[y == cl]) * ((x[y == cl].mean() - grand_mean) ** 2)
        for cl in classes
    ])
    ss_total = sum((x[y.notna()] - grand_mean) ** 2)
    return ss_between / ss_total if ss_total != 0 else 0

numerical_cols = df.columns.drop('Type_of_glass')
etas = {col: eta_squared(df[col], df['Type_of_glass']) for col in numerical_cols}
etas_series = pd.Series(etas)

# 4. Normalizar e aplicar pesos Eta²
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df[numerical_cols]), columns=numerical_cols)
df_weighted = df_scaled * etas_series

# 5. Função de imputação com KNN usando distância euclidiana
def knn_impute(row_idx, df_weighted, df_target, k=5):
    row = df_weighted.iloc[row_idx].values.reshape(1, -1)

    # Seleciona apenas amostras conhecidas
    df_known = df_weighted[df_target.notna()]
    target_known = df_target[df_target.notna()]

    if df_known.empty:
        return target_known.mode().iloc[0]

    distances = cdist(row, df_known.values, metric='euclidean').flatten()
    nearest_indices = df_known.index[np.argsort(distances)[:k]]
    nearest_labels = target_known.loc[nearest_indices].dropna()

    if nearest_labels.empty:
        return target_known.mode().iloc[0]

    return Counter(nearest_labels).most_common(1)[0][0]

# 6. Aplicar imputação
df_imputed = df['Type_of_glass'].copy()
missing_mask = df_imputed.isna()

for idx in df[missing_mask].index:
    df_imputed[idx] = knn_impute(idx, df_weighted, df['Type_of_glass'])

# 7. Avaliação da Imputação
true_values = df_original.loc[missing_indices, 'Type_of_glass']
predicted_values = df_imputed.loc[missing_indices]
acuracia = accuracy_score(true_values, predicted_values)

print(f"Taxa de acerto da imputação: {acuracia:.4f} ({acuracia * 100:.2f}%)")



Taxa de acerto da imputação: 0.7143 (71.43%)


#### Pipeline Alternativa: KNN Simples (Sem Eta²)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import cdist
from collections import Counter
import random

# 1. Carregar e preparar os dados
file_path = 'data/glass.data'
columns = ['Id', 'RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'Type_of_glass']

df = pd.read_csv(file_path, header=None, names=columns)
df.drop(columns=['Id'], inplace=True)

# Salvar os dados originais para avaliação
df_original = df.copy()

random.seed(42)

# 2. Inserir valores ausentes (MCAR) em Type_of_glass
missing_frac = 0.2
n_missing = int(missing_frac * len(df))
missing_indices = random.sample(list(df.index), n_missing)
df.loc[missing_indices, 'Type_of_glass'] = np.nan

# 3. Normalizar os dados (sem ponderar por Eta²)
numerical_cols = df.columns.drop('Type_of_glass')
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df[numerical_cols]), columns=numerical_cols)

# 4. Função de imputação com KNN simples (não ponderado)
def knn_impute_simple(row_idx, df_scaled, df_target, k=5):
    row = df_scaled.iloc[row_idx].values.reshape(1, -1)

    df_known = df_scaled[df_target.notna()]
    target_known = df_target[df_target.notna()]

    if df_known.empty:
        return target_known.mode().iloc[0]

    distances = cdist(row, df_known.values, metric='euclidean').flatten()
    nearest_indices = df_known.index[np.argsort(distances)[:k]]
    nearest_labels = target_known.loc[nearest_indices].dropna()

    if nearest_labels.empty:
        return target_known.mode().iloc[0]

    return Counter(nearest_labels).most_common(1)[0][0]

# 5. Aplicar imputação
df_imputed_simple = df['Type_of_glass'].copy()
missing_mask = df_imputed_simple.isna()

for idx in df[missing_mask].index:
    df_imputed_simple[idx] = knn_impute_simple(idx, df_scaled, df['Type_of_glass'])

# 6. Avaliação da Imputação
true_values = df_original.loc[missing_indices, 'Type_of_glass']
predicted_values_simple = df_imputed_simple.loc[missing_indices]
acuracia_simple = accuracy_score(true_values, predicted_values_simple)

print(f"Taxa de acerto com KNN simples (sem Eta²): {acuracia_simple:.4f} ({acuracia_simple * 100:.2f}%)")


Taxa de acerto com KNN simples (sem Eta²): 0.6905 (69.05%)


### KNN com Gower

In [56]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from collections import Counter
import random

# 1. Carregar e preparar os dados
file_path = 'data/glass.data'
columns = ['Id', 'RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'Type_of_glass']

df = pd.read_csv(file_path, header=None, names=columns)
df.drop(columns=['Id'], inplace=True)

# Salvar os dados originais para avaliação
df_original = df.copy()

random.seed(42)

# 2. Inserir valores ausentes (MCAR) em Type_of_glass
missing_frac = 0.2
n_missing = int(missing_frac * len(df))
missing_indices = random.sample(list(df.index), n_missing)
df.loc[missing_indices, 'Type_of_glass'] = np.nan

# 3. Calcular distância de Gower (numérica apenas)
def gower_distance_matrix(X):
    X = X.copy()
    ranges = X.max() - X.min()
    X_norm = (X - X.min()) / ranges
    n = X.shape[0]
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        dist_matrix[i] = np.abs(X_norm.values[i] - X_norm.values).mean(axis=1)
    return dist_matrix

numerical_cols = df.columns.drop('Type_of_glass')
df_numeric = df[numerical_cols]

# Pré-calcular matriz de distâncias Gower (linha x linha)
gower_matrix = gower_distance_matrix(df_numeric)

# 4. Imputação com Gower
def knn_impute_gower(row_idx, gower_matrix, df_target, k=5):
    # Distâncias da linha atual para todas as outras
    distances = gower_matrix[row_idx]
    
    # Excluir os que também estão ausentes
    known_mask = df_target.notna().values
    distances_known = distances[known_mask]
    indices_known = np.where(known_mask)[0]

    if len(indices_known) == 0:
        return df_target.mode().iloc[0]

    # Selecionar os k mais próximos
    nearest_idxs = indices_known[np.argsort(distances_known)[:k]]
    nearest_labels = df_target.iloc[nearest_idxs].dropna()

    if nearest_labels.empty:
        return df_target.mode().iloc[0]

    return Counter(nearest_labels).most_common(1)[0][0]

# 5. Aplicar imputação
df_imputed_gower = df['Type_of_glass'].copy()
missing_mask = df_imputed_gower.isna()

for idx in df[missing_mask].index:
    df_imputed_gower[idx] = knn_impute_gower(idx, gower_matrix, df['Type_of_glass'])

# 6. Avaliação da Imputação
true_values = df_original.loc[missing_indices, 'Type_of_glass']
predicted_values_gower = df_imputed_gower.loc[missing_indices]
acuracia_gower = accuracy_score(true_values, predicted_values_gower)

print(f"Taxa de acerto com KNN + Gower: {acuracia_gower:.4f} ({acuracia_gower * 100:.2f}%)")


Taxa de acerto com KNN + Gower: 0.7143 (71.43%)


### Variando os Seeds

Seed (Semente) do Random:
- É um valor inicial que controla a geração de números aleatórios no código. Definir a seed garante que os resultados sejam reproduzíveis — ou seja, que o processo aleatório aconteça da mesma forma toda vez que rodar.

Por que variar a seed?
- Como a inserção dos valores ausentes é aleatória, diferentes seeds geram diferentes padrões de dados faltantes, impactando a performance da imputação. Avaliar os métodos KNN em várias seeds permite medir a robustez e a consistência de cada técnica, fornecendo uma comparação mais confiável do desempenho médio e sua variação.

In [59]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import cdist
from collections import Counter
import random

# --- Funções auxiliares ---

def eta_squared(x, y):
    classes = y.dropna().unique()
    grand_mean = x[y.notna()].mean()
    ss_between = sum([
        len(x[y == cl]) * ((x[y == cl].mean() - grand_mean) ** 2)
        for cl in classes
    ])
    ss_total = sum((x[y.notna()] - grand_mean) ** 2)
    return ss_between / ss_total if ss_total != 0 else 0

def gower_distance_matrix(X):
    X = X.copy()
    ranges = X.max() - X.min()
    X_norm = (X - X.min()) / ranges
    n = X.shape[0]
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        dist_matrix[i] = np.abs(X_norm.values[i] - X_norm.values).mean(axis=1)
    return dist_matrix

def knn_impute(row_idx, df_weighted, df_target, k=5):
    row = df_weighted.iloc[row_idx].values.reshape(1, -1)
    df_known = df_weighted[df_target.notna()]
    target_known = df_target[df_target.notna()]
    if df_known.empty:
        return target_known.mode().iloc[0]
    distances = cdist(row, df_known.values, metric='euclidean').flatten()
    nearest_indices = df_known.index[np.argsort(distances)[:k]]
    nearest_labels = target_known.loc[nearest_indices].dropna()
    if nearest_labels.empty:
        return target_known.mode().iloc[0]
    return Counter(nearest_labels).most_common(1)[0][0]

def knn_impute_simple(row_idx, df_scaled, df_target, k=5):
    row = df_scaled.iloc[row_idx].values.reshape(1, -1)
    df_known = df_scaled[df_target.notna()]
    target_known = df_target[df_target.notna()]
    if df_known.empty:
        return target_known.mode().iloc[0]
    distances = cdist(row, df_known.values, metric='euclidean').flatten()
    nearest_indices = df_known.index[np.argsort(distances)[:k]]
    nearest_labels = target_known.loc[nearest_indices].dropna()
    if nearest_labels.empty:
        return target_known.mode().iloc[0]
    return Counter(nearest_labels).most_common(1)[0][0]

def knn_impute_gower(row_idx, gower_matrix, df_target, k=5):
    distances = gower_matrix[row_idx]
    known_mask = df_target.notna().values
    distances_known = distances[known_mask]
    indices_known = np.where(known_mask)[0]
    if len(indices_known) == 0:
        return df_target.mode().iloc[0]
    nearest_idxs = indices_known[np.argsort(distances_known)[:k]]
    nearest_labels = df_target.iloc[nearest_idxs].dropna()
    if nearest_labels.empty:
        return df_target.mode().iloc[0]
    return Counter(nearest_labels).most_common(1)[0][0]

# --- Pipeline que executa um experimento com um seed fixo ---

def run_experiment(seed, missing_frac=0.2, k=5):
    # 1. Carregar e preparar os dados
    file_path = 'data/glass.data'
    columns = ['Id', 'RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe', 'Type_of_glass']
    df = pd.read_csv(file_path, header=None, names=columns)
    df.drop(columns=['Id'], inplace=True)
    df_original = df.copy()

    # Fixar seed para reproducibilidade do missing
    random.seed(seed)

    # 2. Inserir valores ausentes MCAR
    n_missing = int(missing_frac * len(df))
    missing_indices = random.sample(list(df.index), n_missing)
    df.loc[missing_indices, 'Type_of_glass'] = np.nan

    # Colunas numéricas
    numerical_cols = df.columns.drop('Type_of_glass')

    # 3. KNN com Eta² ponderado + Euclidiana
    etas = {col: eta_squared(df[col], df['Type_of_glass']) for col in numerical_cols}
    etas_series = pd.Series(etas)
    scaler = MinMaxScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df[numerical_cols]), columns=numerical_cols)
    df_weighted = df_scaled * etas_series

    df_imputed = df['Type_of_glass'].copy()
    for idx in df_imputed[df_imputed.isna()].index:
        df_imputed[idx] = knn_impute(idx, df_weighted, df['Type_of_glass'], k=k)

    acc_weighted = accuracy_score(df_original.loc[missing_indices, 'Type_of_glass'], df_imputed.loc[missing_indices])

    # 4. KNN simples (sem Eta²) + Euclidiana
    df_scaled_simple = pd.DataFrame(scaler.fit_transform(df[numerical_cols]), columns=numerical_cols)

    df_imputed_simple = df['Type_of_glass'].copy()
    for idx in df_imputed_simple[df_imputed_simple.isna()].index:
        df_imputed_simple[idx] = knn_impute_simple(idx, df_scaled_simple, df['Type_of_glass'], k=k)

    acc_simple = accuracy_score(df_original.loc[missing_indices, 'Type_of_glass'], df_imputed_simple.loc[missing_indices])

    # 5. KNN com distância de Gower
    df_numeric = df[numerical_cols]
    gower_matrix = gower_distance_matrix(df_numeric)

    df_imputed_gower = df['Type_of_glass'].copy()
    for idx in df_imputed_gower[df_imputed_gower.isna()].index:
        df_imputed_gower[idx] = knn_impute_gower(idx, gower_matrix, df['Type_of_glass'], k=k)

    acc_gower = accuracy_score(df_original.loc[missing_indices, 'Type_of_glass'], df_imputed_gower.loc[missing_indices])

    return acc_weighted, acc_simple, acc_gower

# --- Executar vários experimentos ---

seeds = list(range(10, 40))  # 30 execuções, seeds de 10 a 39
results = []

for s in seeds:
    print(f"Executando seed {s}...")
    acc_w, acc_s, acc_g = run_experiment(s)
    results.append({'seed': s, 'weighted': acc_w, 'simple': acc_s, 'gower': acc_g})

df_results = pd.DataFrame(results)

# --- Resultados resumidos ---
summary = df_results.describe().loc[['mean', 'std']]
print("\nResumo dos resultados (média e std das acurácias):")
print(summary)

# --- Você pode ainda exportar os resultados para CSV para análises futuras ---
# df_results.to_csv('knn_imputation_comparison_results.csv', index=False)


Executando seed 10...
Executando seed 11...
Executando seed 12...
Executando seed 13...
Executando seed 14...
Executando seed 15...
Executando seed 16...
Executando seed 17...
Executando seed 18...
Executando seed 19...
Executando seed 20...
Executando seed 21...
Executando seed 22...
Executando seed 23...
Executando seed 24...
Executando seed 25...
Executando seed 26...
Executando seed 27...
Executando seed 28...
Executando seed 29...
Executando seed 30...
Executando seed 31...
Executando seed 32...
Executando seed 33...
Executando seed 34...
Executando seed 35...
Executando seed 36...
Executando seed 37...
Executando seed 38...
Executando seed 39...

Resumo dos resultados (média e std das acurácias):
           seed  weighted    simple     gower
mean  24.500000  0.672222  0.664286  0.706349
std    8.803408  0.065185  0.056741  0.056735


Média (mean):
- É a média da taxa de acerto (accuracy) da imputação para cada método, calculada ao longo das 30 execuções com seeds diferentes.
Exemplo: O método com distância de Gower tem uma média de acurácia de ~70,6%, maior que os outros dois.

Desvio padrão (std):
- Mede a variabilidade das acurácias entre diferentes seeds. Quanto menor o desvio padrão, mais consistente é o desempenho do método em diferentes execuções.
Aqui os desvios padrões são semelhantes entre os métodos, em torno de 5.6% a 6.5%.

Insights:
- Melhor acurácia média: O método KNN + Gower (0.7063) teve a melhor média, sugerindo que usar a distância de Gower ajuda a imputar melhor para esse dataset.

- Consistência: Todos os métodos apresentam uma variação considerável (~5-6% de std), o que mostra que o desempenho depende bastante da amostra gerada (seed).

- Diferenças pequenas entre Weighted e Simple: O método ponderado por Eta² ficou um pouco melhor que o simples, mas essa diferença não é tão grande.

- Importância de testar vários seeds: Como a acurácia varia, um único seed pode não ser representativo. Avaliar média e desvio é importante para conclusões confiáveis.